In [1]:
import speech_recognition as sr
import pyttsx3
import librosa
import os
import nltk
from pydub.silence import split_on_silence
from pydub import audio_segment
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn import decomposition, svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier



c:\Users\oskar\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [22]:
#initalize the voice recognizer
encode = encoder(["mama tipp topp tiptop tip top 50 tack krusbär basketspelare"])
r = sr.Recognizer()

#load in audiofile
file= sr.AudioFile( 'tal\oskar_tal_0_1.wav')

#record sound from file with recognizer
with file as source:
    audio = r.record(source)

#use googles text to speech to get text
text = r.recognize_google(audio,show_all=True)['alternative'][0]
print(text['transcript'])
vector = encode.transform([text['transcript']])
encoded_text = vector.toarray()[0]
#print results
print(text)
print(encoded_text)
print("Speech from audio: " + text['transcript'])
print("Accuracy: " +str(text['confidence']))

Mama Tip Top thank you thank you talk to you spell boss gets together
{'transcript': 'Mama Tip Top thank you thank you talk to you spell boss gets together', 'confidence': 0.91319448}
[0 0 0 1 0 1 0 0 1 0]
Speech from audio: Mama Tip Top thank you thank you talk to you spell boss gets together
Accuracy: 0.91319448


In [2]:


'''Functions to detect different conditons in speech'''

# Create a recognizer instance

# Define a function to count the number of pauses in the speech
def count_pauses(file):
    # Split the audio into chunks
    sound = audio_segment.AudioSegment.from_wav(file)
    chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=-40)
    # Count the number of chunks
    num_chunks = len(chunks)
    # Return the number of pauses
    return num_chunks - 1

# Define a function to count the number of repetitions in the speech
def count_repetitions(text):
    # Split the text into words
    words = text.split()
    # Count the number of repetitions
    num_repetitions = 0
    for i in range(len(words)-1):
        if words[i] == words[i+1]:
            num_repetitions += 1
    # Return the number of repetitions
    return num_repetitions


# Define a function to detect signs of aphasia
def detect_aphasia(text):
    # Load a database of common words and phrases
    common_words = ["mama", 'tipp','topp','tiptop', "tip", "top", "50","tack", "krusbär", "basketspelare"]
    # Split the text into words
    words = text.split()
    # Check for any word substitutions or word-finding difficulties
    for word in words:
        if word.lower() not in common_words:
            print(word)
            return True
    # No signs of aphasia detected
    return False

# Define a function to detect signs of babbling, work in progress
def detect_babbling(text):
    # Split the text into words
    words = text.split()
    # Count the number of nonsensical or repetitive sounds
    num_babbles = 0
    for word in words:
        if word.lower() in ["uh", "um", "ah", "uhh", "umm", "ahh", "babble", "gaga", "goo", "gurgling", "cooing"]:
            num_babbles += 1
    # Return True if there are more than 2 nonsensical or repetitive sounds
    return num_babbles > 2

# Define a function to detect signs of stuttering
def detect_stuttering(text):
    # Split the text into words
    words = nltk.word_tokenize(text)
    # Count the number of repetitions or prolongations
    num_stutters = 0
    for i in range(len(words)-1):
        if len(words[i]) > 1:
            if words[i][0].isalpha() and words[i][1].isalpha() and words[i][0].lower() == words[i][1].lower():
                num_stutters += 1
    # Return True if there are more than 2 repetitions or prolongations
    return num_stutters > 2


def encoder(text):
    vectorizer = CountVectorizer()
    vectorizer.fit(text)    
    return vectorizer


# Transcribe the audio file
# with audio_file as source:
#     audio = r.record(source)
#     text = r.recognize_google(audio)
# print(text)
# # Count the number of pauses and repetitions
# num_pauses = count_pauses(audio)
# num_repetitions = count_repetitions(text)

# # Detect signs of aphasia and babbling
# has_aphasia = detect_aphasia(text)
# has_babbling = detect_babbling(text)
# has_stuttering = detect_stuttering(text)

# # Print the results
# print("Number of pauses:", num_pauses)
# print("Number of repetitions:", num_repetitions)

# if has_babbling:
#     print("Signs of babbling detected.")
# else:
#     print("No signs of babbling detected.")
# if has_stuttering:
#     print("Signs of stuttering")


In [3]:
speech_data = []
encode = encoder(["mama tipp topp tiptop tip top 50 tack krusbär basketspelare"])

for file in os.listdir(r'C:\Users\oskar\Documents\repo\stroke-video-analysis\tal'):
    data = []
    r = sr.Recognizer()
    print('Processing: ' + file + '...')
    # Load the audio file
    audio_file = sr.AudioFile('tal\\' + file)

    with audio_file as source:
        audio = r.record(source)
        try:
            text = r.recognize_google(audio,show_all = True, language="sv-SE")['alternative'][0]
            
            vector = encode.transform([text['transcript']])
            v = vector.toarray()[0]
            data = v.tolist()
            data.append(float(text['confidence']))
            text = text['transcript']
            print(text)
        except:
            text = "incoherent"
            vector = encode.transform([text])
            v = vector.toarray()[0]
            data = v.tolist()
            data.append(0)
            print('incoherent or no speech')
    # data.append(count_pauses("tal\\" + file))
    # data.append(detect_aphasia(text))
    # data.append(count_repetitions(text))
    print("num of pauses:" + str(count_pauses("tal\\" + file)))
    print("Aphasia:" + str(detect_aphasia(text)))
    print("Repetitions:" + str(count_repetitions(text)))
    print(vector.toarray()[0])
    speech_data.append(data)
    print(len(data))

Processing: Andrei_tal_0_1.wav...
Mama Tipp Topp 55 tack krusbär basketspelare
num of pauses:3
55
Aphasia:True
Repetitions:0
[0 1 1 1 1 0 1 0 0 1]
11
Processing: Andrei_tal_0_2.wav...
Mama Tipp Topp 55 tack krusbär basketspelare
num of pauses:4
55
Aphasia:True
Repetitions:0
[0 1 1 1 1 0 1 0 0 1]
11
Processing: Andrei_tal_0_3.wav...
Mama Tip Top 55 krusbär basketspelare
num of pauses:1
55
Aphasia:True
Repetitions:0
[0 1 1 1 0 1 0 0 1 0]
11
Processing: Andrei_tal_0_4.wav...
Mama Tipp Topp 550 tack krusbär basketspelare
num of pauses:5
550
Aphasia:True
Repetitions:0
[0 1 1 1 1 0 1 0 0 1]
11
Processing: Andrei_tal_0_5.wav...
Mama Tip Top 55 tack krusbär och ska ett spelare
num of pauses:4
55
Aphasia:True
Repetitions:0
[0 0 1 1 1 1 0 0 1 0]
11
Processing: Andrei_tal_1_1.wav...
55 zaak morgenspelare
num of pauses:7
55
Aphasia:True
Repetitions:0
[0 0 0 0 0 0 0 0 0 0]
11
Processing: Andrei_tal_1_2.wav...
typ
num of pauses:8
typ
Aphasia:True
Repetitions:0
[0 0 0 0 0 0 0 0 0 0]
11
Processing: An

In [37]:
l = [0,0,0,0,0,1,1,1,1,1,1,1,1,1,1]
label = []
for i in range(4):
    label += l

print(speech_data)
X_train, X_test, y_train, y_test = train_test_split(
    speech_data, label, test_size=0.3
)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
# y_train = tf.keras.utils.to_categorical(np.asarray(y_train))
# y_test = tf.keras.utils.to_categorical(np.asarray(y_test))
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
print(len(y_train))



[[0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0.84602195], [0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0.87906647], [0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0.88973498], [0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0.86773342], [0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0.8629607], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.69577742], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.62706888], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.65939498], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0.69712871], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.82417589], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0.92127752], [2, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0.85222226], [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0.89318943], [2, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0.92412162], [0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0.90396452], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 

In [41]:
model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [42]:
model.compile(optimizer='adam', loss= tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

result = model.fit(X_train, y_train, epochs = 5, batch_size=2, validation_data = (X_test,y_test))

Epoch 1/5


C:\Users\oskar\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\backend.py:5016: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


21/21 [==============================] - 0s 6ms/step - loss: 0.6581 - accuracy: 0.7381 - val_loss: 0.5433 - val_accuracy: 0.8889
Epoch 2/5
21/21 [==============================] - 0s 1ms/step - loss: 0.4885 - accuracy: 0.9762 - val_loss: 0.3655 - val_accuracy: 1.0000
Epoch 3/5
21/21 [==============================] - 0s 1ms/step - loss: 0.3136 - accuracy: 0.9762 - val_loss: 0.1932 - val_accuracy: 1.0000
Epoch 4/5
21/21 [==============================] - 0s 1ms/step - loss: 0.1649 - accuracy: 0.9762 - val_loss: 0.0885 - val_accuracy: 1.0000
Epoch 5/5
21/21 [==============================] - 0s 1ms/step - loss: 0.0801 - accuracy: 1.0000 - val_loss: 0.0458 - val_accuracy: 1.0000


In [43]:
score = model.evaluate(X_test, y_test, verbose=0)
test = np.round(model.predict(X_test))
print('Test loss:', score[0])
print('Test accuracy:', score[1])
report = accuracy_score(y_test,test)
nn_score = classification_report(y_test,test)
print(report)
print(nn_score)

Test loss: 0.045824043452739716
Test accuracy: 1.0
1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00        12

    accuracy                           1.00        18
   macro avg       1.00      1.00      1.00        18
weighted avg       1.00      1.00      1.00        18



In [44]:
'''support vector machine'''
rbf = svm.SVC(kernel='poly', gamma=0.5, C=0.1).fit(X_train, y_train)
rbf_pred = rbf.predict(X_test)
svm_score = classification_report(y_test,rbf_pred)
rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print(svm_score)

Accuracy (RBF Kernel):  88.89
              precision    recall  f1-score   support

           0       1.00      0.67      0.80         6
           1       0.86      1.00      0.92        12

    accuracy                           0.89        18
   macro avg       0.93      0.83      0.86        18
weighted avg       0.90      0.89      0.88        18



In [45]:
'''AdaBoost'''
adaboost = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = 10),
    n_estimators = 200
)
adaboost.fit(X_train, y_train)
ada_pred = adaboost.predict(X_test)
ada_score = classification_report(y_test,ada_pred)
ada_accuracy = accuracy_score(y_test, ada_pred)
print('Accuracy (AdaBoost): ', "%.2f" % (ada_accuracy*100))
print(ada_score)

Accuracy (AdaBoost):  94.44
              precision    recall  f1-score   support

           0       1.00      0.83      0.91         6
           1       0.92      1.00      0.96        12

    accuracy                           0.94        18
   macro avg       0.96      0.92      0.93        18
weighted avg       0.95      0.94      0.94        18

